In [1]:
!pip install ganblr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
!pip install liac-arff

  Preparing metadata (setup.py) ... done
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11717 sha256=97075f25c8de58ac4ffc508193c35a1fb3e6d91a3b96c20bf7b6d4486b468aa4
  Stored in directory: /root/.cache/pip/wheels/00/23/31/5e562fce1f95aabe57f2a7320d07433ba1cd152bcde2f6a002
Successfully built liac-arff


In [3]:
#Fix np.NAN compatibility
!sed -i 's/np\.NaN/np.nan/g' /usr/local/lib/python3.11/dist-packages/ganblr/kdb.py

!sed -i 's/np\.NaN/np.nan/g' /usr/local/lib/python3.11/dist-packages/ganblr/pyitlib/pyitlib/discrete_random_variable.py

In [4]:
import pandas as pd
import arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

with open('adult 1.arff','r') as f:
  data = arff.load(f)
df = pd.DataFrame(data['data'], columns=[attr[0] for attr in data['attributes']])
df.replace('?', pd.NA, inplace=True)
df.rename(columns={'class': 'income'}, inplace=True)
# Improved cleaning: only clean strings (ignoring interval range values)
df = df.applymap(lambda x: x.strip(" '\".") if isinstance(x, str) and not any(c in x for c in ('[', ']', '-inf')) else x)
print("Columns: ")
print(df.columns.tolist())
df.head(1)

# Separate features (X) and labels (y) — assuming the label is in the last column
X = df.iloc[:, :-1]  # All columns except the last
y = df.iloc[:, -1]   # The last column (target/output label)

# Perform 80/20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optional: Combine features and labels back into DataFrames if needed
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Encode features
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1, dtype=int)
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test)

# Save category mappings for later
encoded_feature_categories = encoder.categories_

<ipython-input-4-25a0b5f7a11f>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip(" '\".") if isinstance(x, str) and not any(c in x for c in ('[', ']', '-inf')) else x)


Columns: 
['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']


In [3]:
from ganblr.models import GANBLR

# Initialise model
model = GANBLR()

# Fit the model on the cleaned training data
model.fit(X_train, y_train, k=0, epochs=100, batch_size=64)

warmup run:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


611/611 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7582 - loss: 0.5037


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100: G_loss = 3.729281, G_accuracy = 0.860031, D_loss = 0.920145, D_accuracy = 0.750688


...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 98/100: G_loss = 5.941494, G_accuracy = 0.874159, D_loss = 0.583198, D_accuracy = 0.866370


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 99/100: G_loss = 3.905237, G_accuracy = 0.873724, D_loss = 0.568607, D_accuracy = 0.829692


Epoch 100/100: G_loss = 3.812008, G_accuracy = 0.873647, D_loss = 0.973279, D_accuracy = 0.727590


In [4]:
# Generate synthetic data
synthetic_data = model.sample(size=len(X_train), verbose=1)
# synthetic_data = model.sample(1000), verbose=1)

# Convert to DataFrame with original column names
import pandas as pd
synthetic_df = pd.DataFrame(synthetic_data, columns=list(X_train.columns) + ['income'])

# Display first few rows
synthetic_df.head()

  0%|          | 0/15 [00:00<?, ?it/s]

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,'(-inf-21.5]',Never-worked,All,Preschool,'(13.5-14.5]',Separated,Priv-house-serv,Unmarried,Other,Female,'(4401-4668.5]','(1978.5-2168.5]','(34.5-39.5]',South,<=50K
1,'(61.5-67.5]',Self-emp-not-inc,All,1st-4th,'(14.5-inf)',Married-AF-spouse,Tech-support,Wife,Other,Male,'(4668.5-4826]','(2384.5-2450.5]','(34.5-39.5]',Taiwan,>50K
2,'(-inf-21.5]',Never-worked,All,Preschool,'(12.5-13.5]',Married-spouse-absent,Farming-fishing,Husband,Amer-Indian-Eskimo,Female,'(4401-4668.5]','(1978.5-2168.5]','(34.5-39.5]',England,<=50K
3,'(61.5-67.5]',Federal-gov,All,Assoc-voc,'(12.5-13.5]',Separated,Machine-op-inspct,Own-child,Black,Female,'(3120-4243.5]','(1978.5-2168.5]','(61.5-inf)',Peru,<=50K
4,'(-inf-21.5]',Never-worked,All,Preschool,'(12.5-13.5]',Never-married,Armed-Forces,Own-child,Black,Female,'(4973.5-5119]','(1978.5-2168.5]','(49.5-61.5]',India,<=50K


In [5]:
# Generate synthetic data matching real data size
synthetic_data = model.sample(size=len(X_train), verbose=0)

# Rebuild column names with label
synthetic_columns = list(X_train.columns) + ['income']

# Create DataFrame
synthetic_df = pd.DataFrame(synthetic_data, columns=synthetic_columns)

In [6]:
print(f"Real: {X_train.assign(income=y_train).shape}, Synthetic: {synthetic_df.shape}")

Real: (39073, 15), Synthetic: (39073, 15)


In [7]:
def compare_all_distributions(real_df, synthetic_df, top_n=10):
    for column in real_df.columns:
        print(f"\n Feature: '{column}'")

        real_dist = real_df[column].value_counts(normalize=True).head(top_n)
        synthetic_dist = synthetic_df[column].value_counts(normalize=True).head(top_n)

        print("\nReal:")
        print(real_dist)

        print("\nSynthetic:")
        print(synthetic_dist)

        print("-" * 40)

# Run comparison
compare_all_distributions(X_train.assign(income=y_train), synthetic_df)


 Feature: 'age'

Real:
age
'(41.5-54.5]'    0.246206
'(35.5-41.5]'    0.154813
'(30.5-35.5]'    0.132956
'(-inf-21.5]'    0.097331
'(27.5-30.5]'    0.077905
'(54.5-61.5]'    0.076242
'(24.5-27.5]'    0.073043
'(21.5-23.5]'    0.051237
'(61.5-67.5]'    0.037647
'(67.5-inf)'     0.027385
Name: proportion, dtype: float64

Synthetic:
age
'(-inf-21.5]'    0.259361
'(21.5-23.5]'    0.107952
'(23.5-24.5]'    0.105239
'(24.5-27.5]'    0.086607
'(67.5-inf)'     0.069588
'(30.5-35.5]'    0.068027
'(54.5-61.5]'    0.064827
'(27.5-30.5]'    0.062652
'(35.5-41.5]'    0.062422
'(61.5-67.5]'    0.057431
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'workclass'

Real:
workclass
Private             0.695544
Self-emp-not-inc    0.077650
Local-gov           0.064213
missing             0.056868
State-gov           0.040821
Self-emp-inc        0.035267
Federal-gov         0.028971
Without-pay         0.000435
Never-worked        0.000230
Name: proportion, dtype: flo

In [8]:
# Logistic Regression
accuracy_lr = model.evaluate(X_test.values, y_test.values, model='lr')
print(f"TSTR Accuracy (LR): {accuracy_lr:.4f}")

# Multi-Layer Perceptron
accuracy_mlp = model.evaluate(X_test.values, y_test.values, model='mlp')
print(f"TSTR Accuracy (MLP): {accuracy_mlp:.4f}")

# Random Forest
accuracy_rf = model.evaluate(X_test.values, y_test.values, model='rf')
print(f"TSTR Accuracy (RF): {accuracy_rf:.4f}")

from xgboost import XGBClassifier

# Create an XGBoost classifier instance
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Pass the instance directly
accuracy_xgbt = model.evaluate(X_test.values, y_test.values, model=xgb_model)
print(f"TSTR Accuracy (XGBT): {accuracy_xgbt:.4f}")

  0%|          | 0/15 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


TSTR Accuracy (LR): 0.4976


  0%|          | 0/15 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


TSTR Accuracy (RF): 0.7238


  0%|          | 0/15 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [04:01:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


TSTR Accuracy (XGBT): 0.6941


In [9]:
import pandas as pd
import arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

with open('nursery 1.arff','r') as f:
  data = arff.load(f)
df = pd.DataFrame(data['data'], columns=[attr[0] for attr in data['attributes']])
df.replace('?', pd.NA, inplace=True)
df.rename(columns={'class': 'status'}, inplace=True)
# Improved cleaning: only clean strings (ignoring interval range values)
df = df.applymap(lambda x: x.strip(" '\".") if isinstance(x, str) and not any(c in x for c in ('[', ']', '-inf')) else x)
print("Columns: ")
print(df.columns.tolist())
df.head(1)

# Separate features (X) and labels (y) — assuming the label is in the last column
X = df.iloc[:, :-1]  # All columns except the last
y = df.iloc[:, -1]   # The last column (target/output label)

# Perform 80/20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optional: Combine features and labels back into DataFrames if needed
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Encode features
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1, dtype=int)
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test)

# Save category mappings for later
encoded_feature_categories = encoder.categories_

<ipython-input-9-75b39a0d9d76>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip(" '\".") if isinstance(x, str) and not any(c in x for c in ('[', ']', '-inf')) else x)


Columns: 
['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'status']


In [10]:
from ganblr.models import GANBLR

# Initialise model
model = GANBLR()

# Fit the model on the cleaned training data
model.fit(X_train, y_train, k=0, epochs=100, batch_size=64)

warmup run:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4157 - loss: 1.2862


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100: G_loss = 1.765400, G_accuracy = 0.639468, D_loss = 0.887651, D_accuracy = 0.474319


...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 98/100: G_loss = 1.072010, G_accuracy = 0.926601, D_loss = 1.071962, D_accuracy = 0.464975


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 99/100: G_loss = 0.971662, G_accuracy = 0.926022, D_loss = 0.799138, D_accuracy = 0.529419


Epoch 100/100: G_loss = 0.950238, G_accuracy = 0.926890, D_loss = 0.848041, D_accuracy = 0.480950


In [11]:
# Generate synthetic data
synthetic_data = model.sample(size=len(X_train), verbose=1)
# synthetic_data = model.sample(1000), verbose=1)

# Convert to DataFrame with original column names
import pandas as pd
synthetic_df = pd.DataFrame(synthetic_data, columns=list(X_train.columns) + ['status'])

# Display first few rows
synthetic_df.head()

  0%|          | 0/9 [00:00<?, ?it/s]

,parents,has_nurs,form,children,housing,finance,social,health,status
0,usual,less_proper,incomplete,2,convenient,inconv,slightly_prob,priority,priority
1,great_pret,very_crit,foster,3,critical,inconv,problematic,priority,spec_prior
2,usual,less_proper,completed,1,convenient,inconv,nonprob,priority,priority
3,great_pret,critical,foster,3,critical,inconv,problematic,priority,spec_prior
4,great_pret,very_crit,foster,2,less_conv,inconv,problematic,priority,spec_prior


In [12]:
# Generate synthetic data matching real data size
synthetic_data = model.sample(size=len(X_train), verbose=0)

# Rebuild column names with label
synthetic_columns = list(X_train.columns) + ['status']

# Create DataFrame
synthetic_df = pd.DataFrame(synthetic_data, columns=synthetic_columns)

In [13]:
print(f"Real: {X_train.assign(status=y_train).shape}, Synthetic: {synthetic_df.shape}")

Real: (10368, 9), Synthetic: (10368, 9)


In [14]:
def compare_all_distributions(real_df, synthetic_df, top_n=10):
    for column in real_df.columns:
        print(f"\n Feature: '{column}'")

        real_dist = real_df[column].value_counts(normalize=True).head(top_n)
        synthetic_dist = synthetic_df[column].value_counts(normalize=True).head(top_n)

        print("\nReal:")
        print(real_dist)

        print("\nSynthetic:")
        print(synthetic_dist)

        print("-" * 40)

# Run comparison
compare_all_distributions(X_train.assign(status=y_train), synthetic_df)


 Feature: 'parents'

Real:
parents
great_pret     0.335552
pretentious    0.334201
usual          0.330247
Name: proportion, dtype: float64

Synthetic:
parents
great_pret     0.450617
usual          0.325617
pretentious    0.223765
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'has_nurs'

Real:
has_nurs
less_proper    0.201775
improper       0.200617
critical       0.200424
very_crit      0.198978
proper         0.198206
Name: proportion, dtype: float64

Synthetic:
has_nurs
very_crit      0.302276
less_proper    0.240066
proper         0.181617
critical       0.161458
improper       0.114583
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'form'

Real:
form
complete      0.253665
completed     0.249421
foster        0.248843
incomplete    0.248071
Name: proportion, dtype: float64

Synthetic:
form
foster        0.311535
incomplete    0.241223
complete      0.231096
completed     0.216146
Name: proportion, dtype

In [15]:
X_test_filtered = X_test[y_test != 'recommend']
y_test_filtered = y_test[y_test != 'recommend']


# Logistic Regression
accuracy_lr = model.evaluate(X_test_filtered.values, y_test_filtered.values, model='lr')
print(f"TSTR Accuracy (LR): {accuracy_lr:.4f}")

# Multi-Layer Perceptron
accuracy_mlp = model.evaluate(X_test_filtered.values, y_test_filtered.values, model='mlp')
print(f"TSTR Accuracy (MLP): {accuracy_mlp:.4f}")

# Random Forest
accuracy_rf = model.evaluate(X_test_filtered.values, y_test_filtered.values, model='rf')
print(f"TSTR Accuracy (RF): {accuracy_rf:.4f}")

from xgboost import XGBClassifier

# Create an XGBoost classifier instance
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Pass the instance directly
accuracy_xgbt = model.evaluate(X_test_filtered.values, y_test_filtered.values, model=xgb_model)
print(f"TSTR Accuracy (XGBT): {accuracy_xgbt:.4f}")

  0%|          | 0/9 [00:00<?, ?it/s]

TSTR Accuracy (LR): 0.7965


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


  0%|          | 0/9 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


TSTR Accuracy (RF): 0.7915


  0%|          | 0/9 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [04:14:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


TSTR Accuracy (XGBT): 0.7985


In [5]:
import pandas as pd
import arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

with open('connect-4.arff','r') as f:
  data = arff.load(f)
df = pd.DataFrame(data['data'], columns=[attr[0] for attr in data['attributes']])
df.replace('?', pd.NA, inplace=True)
df.rename(columns={'class': 'predic'}, inplace=True)
# Improved cleaning: only clean strings (ignoring interval range values)
df = df.applymap(lambda x: x.strip(" '\".") if isinstance(x, str) and not any(c in x for c in ('[', ']', '-inf')) else x)
print("Columns: ")
print(df.columns.tolist())
df.head(1)

# Separate features (X) and labels (y) — assuming the label is in the last column
X = df.iloc[:, :-1]  # All columns except the last
y = df.iloc[:, -1]   # The last column (target/output label)

# Perform 80/20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optional: Combine features and labels back into DataFrames if needed
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Encode features
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1, dtype=int)
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test)

# Save category mappings for later
encoded_feature_categories = encoder.categories_

<ipython-input-5-78cd7a53c6a9>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip(" '\".") if isinstance(x, str) and not any(c in x for c in ('[', ']', '-inf')) else x)


Columns: 
['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'g1', 'g2', 'g3', 'g4', 'g5', 'g6', 'predic']


In [6]:
from ganblr.models import GANBLR

# Initialise model
model = GANBLR()

# Fit the model on the cleaned training data
model.fit(X_train, y_train, k=0, epochs=150, batch_size=64)

warmup run:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


845/845 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.6740 - loss: 0.8250


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150: G_loss = 4.081019, G_accuracy = 0.744084, D_loss = 0.161754, D_accuracy = 0.951499


...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 148/150: G_loss = 3.554441, G_accuracy = 0.759423, D_loss = 0.222939, D_accuracy = 0.927849


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 149/150: G_loss = 3.881813, G_accuracy = 0.759330, D_loss = 0.165622, D_accuracy = 0.954841


Epoch 150/150: G_loss = 3.490303, G_accuracy = 0.759404, D_loss = 0.258806, D_accuracy = 0.901286


In [7]:
# Generate synthetic data
synthetic_data = model.sample(size=len(X_train), verbose=1)
# synthetic_data = model.sample(1000), verbose=1)

# Convert to DataFrame with original column names
import pandas as pd
synthetic_df = pd.DataFrame(synthetic_data, columns=list(X_train.columns) + ['predic'])

# Display first few rows
synthetic_df.head()

  0%|          | 0/43 [00:00<?, ?it/s]

,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,...,f4,f5,f6,g1,g2,g3,g4,g5,g6,predic
0,b,b,b,o,x,b,x,x,x,x,...,b,o,x,o,b,o,x,b,b,win
1,x,b,x,x,x,x,o,b,o,b,...,b,o,b,b,b,x,x,b,b,loss
2,x,o,x,b,b,b,b,x,o,x,...,b,x,x,b,x,o,x,x,x,loss
3,b,o,o,b,b,o,b,o,o,o,...,b,b,x,b,x,x,o,x,b,draw
4,b,x,o,o,b,b,x,b,x,o,...,x,o,o,b,o,o,x,o,b,win


In [8]:
# Generate synthetic data matching real data size
synthetic_data = model.sample(size=len(X_train), verbose=0)

# Rebuild column names with label
synthetic_columns = list(X_train.columns) + ['predic']

# Create DataFrame
synthetic_df = pd.DataFrame(synthetic_data, columns=synthetic_columns)

In [9]:
print(f"Real: {X_train.assign(status=y_train).shape}, Synthetic: {synthetic_df.shape}")

Real: (54045, 43), Synthetic: (54045, 43)


In [10]:
def compare_all_distributions(real_df, synthetic_df, top_n=10):
    for column in real_df.columns:
        print(f"\n Feature: '{column}'")

        real_dist = real_df[column].value_counts(normalize=True).head(top_n)
        synthetic_dist = synthetic_df[column].value_counts(normalize=True).head(top_n)

        print("\nReal:")
        print(real_dist)

        print("\nSynthetic:")
        print(synthetic_dist)

        print("-" * 40)

# Run comparison
compare_all_distributions(X_train.assign(predic=y_train), synthetic_df)


 Feature: 'a1'

Real:
a1
b    0.370506
x    0.329559
o    0.299935
Name: proportion, dtype: float64

Synthetic:
a1
o    0.407364
b    0.304654
x    0.287982
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'a2'

Real:
a2
b    0.642687
x    0.179369
o    0.177944
Name: proportion, dtype: float64

Synthetic:
a2
o    0.403386
b    0.332038
x    0.264576
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'a3'

Real:
a3
b    0.818966
o    0.092664
x    0.088371
Name: proportion, dtype: float64

Synthetic:
a3
o    0.355796
b    0.352503
x    0.291701
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'a4'

Real:
a4
b    0.911944
o    0.045925
x    0.042132
Name: proportion, dtype: float64

Synthetic:
a4
b    0.363197
o    0.333759
x    0.303044
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'a5'

Real:
a5
b    0.966250
o    0.016875
x    0.016875
Name

In [11]:
# Logistic Regression
accuracy_lr = model.evaluate(X_test.values, y_test.values, model='lr')
print(f"TSTR Accuracy (LR): {accuracy_lr:.4f}")

# Multi-Layer Perceptron
accuracy_mlp = model.evaluate(X_test.values, y_test.values, model='mlp')
print(f"TSTR Accuracy (MLP): {accuracy_mlp:.4f}")

# Random Forest
accuracy_rf = model.evaluate(X_test.values, y_test.values, model='rf')
print(f"TSTR Accuracy (RF): {accuracy_rf:.4f}")

from xgboost import XGBClassifier

# Create an XGBoost classifier instance
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Pass the instance directly
accuracy_xgbt = model.evaluate(X_test.values, y_test.values, model=xgb_model)
print(f"TSTR Accuracy (XGBT): {accuracy_xgbt:.4f}")

  0%|          | 0/43 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


TSTR Accuracy (LR): 0.7464


  0%|          | 0/43 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


TSTR Accuracy (RF): 0.6613


  0%|          | 0/43 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [05:51:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


TSTR Accuracy (XGBT): 0.7305


In [12]:
import pandas as pd
import arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

with open('car 1.arff','r') as f:
  data = arff.load(f)
df = pd.DataFrame(data['data'], columns=[attr[0] for attr in data['attributes']])
df.replace('?', pd.NA, inplace=True)
df.rename(columns={'class': 'option'}, inplace=True)
# Improved cleaning: only clean strings (ignoring interval range values)
df = df.applymap(lambda x: x.strip(" '\".") if isinstance(x, str) and not any(c in x for c in ('[', ']', '-inf')) else x)
print("Columns: ")
print(df.columns.tolist())
df.head(1)

# Separate features (X) and labels (y) — assuming the label is in the last column
X = df.iloc[:, :-1]  # All columns except the last
y = df.iloc[:, -1]   # The last column (target/output label)

# Perform 80/20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optional: Combine features and labels back into DataFrames if needed
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Encode features
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1, dtype=int)
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test)

# Save category mappings for later
encoded_feature_categories = encoder.categories_

Columns: 
['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'option']


<ipython-input-12-a5bd96f47247>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip(" '\".") if isinstance(x, str) and not any(c in x for c in ('[', ']', '-inf')) else x)


In [14]:
from ganblr.models import GANBLR

# Initialise model
model = GANBLR()

# Fit the model on the cleaned training data
model.fit(X_train, y_train, k=0, epochs=100, batch_size=64)

warmup run:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2366 - loss: 1.5081   


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100: G_loss = 1.699978, G_accuracy = 0.280029, D_loss = 1.079384, D_accuracy = 0.492990


...

Epoch 97/100: G_loss = 3.371823, G_accuracy = 0.863965, D_loss = 1.647313, D_accuracy = 0.497512
Epoch 98/100: G_loss = 4.389671, G_accuracy = 0.865412, D_loss = 2.181657, D_accuracy = 0.497965


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 99/100: G_loss = 1.012561, G_accuracy = 0.866860, D_loss = 0.947108, D_accuracy = 0.505654


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 100/100: G_loss = 1.208887, G_accuracy = 0.869030, D_loss = 0.991830, D_accuracy = 0.495703


In [15]:
# Generate synthetic data
synthetic_data = model.sample(size=len(X_train), verbose=1)
# synthetic_data = model.sample(1000), verbose=1)

# Convert to DataFrame with original column names
import pandas as pd
synthetic_df = pd.DataFrame(synthetic_data, columns=list(X_train.columns) + ['option'])

# Display first few rows
synthetic_df.head()

  0%|          | 0/7 [00:00<?, ?it/s]

,buying,maint,doors,persons,lug_boot,safety,option
0,med,low,2,2,big,low,unacc
1,low,low,3,2,small,low,unacc
2,high,low,5more,2,small,low,unacc
3,vhigh,vhigh,2,2,small,low,unacc
4,vhigh,vhigh,2,4,med,low,unacc


In [16]:
# Generate synthetic data matching real data size
synthetic_data = model.sample(size=len(X_train), verbose=0)

# Rebuild column names with label
synthetic_columns = list(X_train.columns) + ['option']

# Create DataFrame
synthetic_df = pd.DataFrame(synthetic_data, columns=synthetic_columns)

In [17]:
print(f"Real: {X_train.assign(status=y_train).shape}, Synthetic: {synthetic_df.shape}")

Real: (1382, 7), Synthetic: (1382, 7)


In [18]:
def compare_all_distributions(real_df, synthetic_df, top_n=10):
    for column in real_df.columns:
        print(f"\n Feature: '{column}'")

        real_dist = real_df[column].value_counts(normalize=True).head(top_n)
        synthetic_dist = synthetic_df[column].value_counts(normalize=True).head(top_n)

        print("\nReal:")
        print(real_dist)

        print("\nSynthetic:")
        print(synthetic_dist)

        print("-" * 40)

# Run comparison
compare_all_distributions(X_train.assign(option=y_train), synthetic_df)


 Feature: 'buying'

Real:
buying
med      0.256874
low      0.252533
high     0.246020
vhigh    0.244573
Name: proportion, dtype: float64

Synthetic:
buying
vhigh    0.356729
low      0.256151
high     0.234443
med      0.152677
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'maint'

Real:
maint
high     0.258321
med      0.254703
vhigh    0.245297
low      0.241679
Name: proportion, dtype: float64

Synthetic:
maint
vhigh    0.308973
high     0.267728
med      0.239508
low      0.183792
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'doors'

Real:
doors
2        0.252533
4        0.250362
5more    0.249638
3        0.247467
Name: proportion, dtype: float64

Synthetic:
doors
5more    0.299566
3        0.247467
2        0.232272
4        0.220695
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'persons'

Real:
persons
4       0.342258
more    0.332127
2       0.325615
Name: p

In [19]:
# Logistic Regression
accuracy_lr = model.evaluate(X_test.values, y_test.values, model='lr')
print(f"TSTR Accuracy (LR): {accuracy_lr:.4f}")

# Multi-Layer Perceptron
accuracy_mlp = model.evaluate(X_test.values, y_test.values, model='mlp')
print(f"TSTR Accuracy (MLP): {accuracy_mlp:.4f}")

# Random Forest
accuracy_rf = model.evaluate(X_test.values, y_test.values, model='rf')
print(f"TSTR Accuracy (RF): {accuracy_rf:.4f}")

from xgboost import XGBClassifier

# Create an XGBoost classifier instance
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Pass the instance directly
accuracy_xgbt = model.evaluate(X_test.values, y_test.values, model=xgb_model)
print(f"TSTR Accuracy (XGBT): {accuracy_xgbt:.4f}")

  0%|          | 0/7 [00:00<?, ?it/s]

TSTR Accuracy (LR): 0.8266


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


  0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


TSTR Accuracy (RF): 0.7717


  0%|          | 0/7 [00:00<?, ?it/s]

TSTR Accuracy (XGBT): 0.7341


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:06:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [20]:
import pandas as pd
import arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

with open('chess.arff','r') as f:
  data = arff.load(f)
df = pd.DataFrame(data['data'], columns=[attr[0] for attr in data['attributes']])
df.replace('?', pd.NA, inplace=True)
df.rename(columns={'class': 'result'}, inplace=True)
# Improved cleaning: only clean strings (ignoring interval range values)
df = df.applymap(lambda x: x.strip(" '\".") if isinstance(x, str) and not any(c in x for c in ('[', ']', '-inf')) else x)
print("Columns: ")
print(df.columns.tolist())
df.head(1)

# Separate features (X) and labels (y) — assuming the label is in the last column
X = df.iloc[:, :-1]  # All columns except the last
y = df.iloc[:, -1]   # The last column (target/output label)

# Perform 80/20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optional: Combine features and labels back into DataFrames if needed
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Encode features
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1, dtype=int)
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test)

# Save category mappings for later
encoded_feature_categories = encoder.categories_

Columns: 
['A0', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'result']


<ipython-input-20-afc6c4d83a54>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip(" '\".") if isinstance(x, str) and not any(c in x for c in ('[', ']', '-inf')) else x)


In [21]:
from ganblr.models import GANBLR

# Initialise model
model = GANBLR()

# Fit the model on the cleaned training data
model.fit(X_train, y_train, k=0, epochs=100, batch_size=64)

warmup run:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3034 - loss: 1.0243


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100: G_loss = 1.489283, G_accuracy = 0.356818, D_loss = 1.172289, D_accuracy = 0.227273


...

Epoch 98/100: G_loss = 1.044277, G_accuracy = 0.909091, D_loss = 0.725934, D_accuracy = 0.546875


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 99/100: G_loss = 1.182628, G_accuracy = 0.909091, D_loss = 0.800827, D_accuracy = 0.532670


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 100/100: G_loss = 0.923997, G_accuracy = 0.911364, D_loss = 0.724134, D_accuracy = 0.465909


In [22]:
# Generate synthetic data
synthetic_data = model.sample(size=len(X_train), verbose=1)
# synthetic_data = model.sample(1000), verbose=1)

# Convert to DataFrame with original column names
import pandas as pd
synthetic_df = pd.DataFrame(synthetic_data, columns=list(X_train.columns) + ['result'])

# Display first few rows
synthetic_df.head()

  0%|          | 0/40 [00:00<?, ?it/s]

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A30,A31,A32,A33,A34,A35,A36,A37,A38,result
0,1,1,1,1,2,1,2,2,2,1,...,2,2,1,2,2,2,1,2,2,1
1,2,1,1,2,2,2,2,1,1,2,...,1,2,1,2,2,1,1,2,1,1
2,1,2,1,2,2,2,1,2,2,2,...,1,2,1,1,2,2,1,2,1,1
3,2,2,2,1,1,2,1,2,1,1,...,1,1,1,1,2,1,2,1,1,1
4,2,1,1,2,2,2,2,2,1,1,...,1,1,1,2,2,1,1,2,2,1


In [23]:
# Generate synthetic data matching real data size
synthetic_data = model.sample(size=len(X_train), verbose=0)

# Rebuild column names with label
synthetic_columns = list(X_train.columns) + ['result']

# Create DataFrame
synthetic_df = pd.DataFrame(synthetic_data, columns=synthetic_columns)

In [24]:
print("Unique labels in y_train:", y_train.unique())
print("Unique labels in y_test:", y_test.unique())

Unique labels in y_train: ['1' '0']
Unique labels in y_test: ['1' '0']


In [25]:
print(f"Real: {X_train.assign(status=y_train).shape}, Synthetic: {synthetic_df.shape}")

Real: (440, 40), Synthetic: (440, 40)


In [26]:
def compare_all_distributions(real_df, synthetic_df, top_n=10):
    for column in real_df.columns:
        print(f"\n Feature: '{column}'")

        real_dist = real_df[column].value_counts(normalize=True).head(top_n)
        synthetic_dist = synthetic_df[column].value_counts(normalize=True).head(top_n)

        print("\nReal:")
        print(real_dist)

        print("\nSynthetic:")
        print(synthetic_dist)

        print("-" * 40)

# Run comparison
compare_all_distributions(X_train.assign(result=y_train), synthetic_df)


 Feature: 'A0'

Real:
A0
1    0.720455
2    0.279545
Name: proportion, dtype: float64

Synthetic:
A0
1    0.538636
2    0.461364
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'A1'

Real:
A1
1    0.711364
2    0.288636
Name: proportion, dtype: float64

Synthetic:
A1
2    0.509091
1    0.490909
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'A2'

Real:
A2
1    0.609091
2    0.390909
Name: proportion, dtype: float64

Synthetic:
A2
2    0.572727
1    0.427273
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'A3'

Real:
A3
2    0.679545
1    0.320455
Name: proportion, dtype: float64

Synthetic:
A3
1    0.556818
2    0.443182
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'A4'

Real:
A4
1    0.8
2    0.2
Name: proportion, dtype: float64

Synthetic:
A4
2    0.631818
1    0.368182
Name: proportion, dtype: float64
-------------------------------

In [27]:
# Logistic Regression
accuracy_lr = model.evaluate(X_test.values, y_test.values, model='lr')
print(f"TSTR Accuracy (LR): {accuracy_lr:.4f}")

# Multi-Layer Perceptron
accuracy_mlp = model.evaluate(X_test.values, y_test.values, model='mlp')
print(f"TSTR Accuracy (MLP): {accuracy_mlp:.4f}")

# Random Forest
accuracy_rf = model.evaluate(X_test.values, y_test.values, model='rf')
print(f"TSTR Accuracy (RF): {accuracy_rf:.4f}")

from xgboost import XGBClassifier

# Create an XGBoost classifier instance
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Pass the instance directly
accuracy_xgbt = model.evaluate(X_test.values, y_test.values, model=xgb_model)
print(f"TSTR Accuracy (XGBT): {accuracy_xgbt:.4f}")

  0%|          | 0/40 [00:00<?, ?it/s]

TSTR Accuracy (LR): 0.7477


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


  0%|          | 0/40 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


TSTR Accuracy (RF): 0.7297


  0%|          | 0/40 [00:00<?, ?it/s]

TSTR Accuracy (XGBT): 0.7568


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:28:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [28]:
import pandas as pd
import arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

with open('credit-a.arff','r') as f:
  data = arff.load(f)
df = pd.DataFrame(data['data'], columns=[attr[0] for attr in data['attributes']])
df.replace('?', pd.NA, inplace=True)
df.rename(columns={'class': 'limit'}, inplace=True)
# Improved cleaning: only clean strings (ignoring interval range values)
df = df.applymap(lambda x: x.strip(" '\".") if isinstance(x, str) and not any(c in x for c in ('[', ']', '-inf')) else x)
print("Columns: ")
print(df.columns.tolist())
df.head(1)

# Separate features (X) and labels (y) — assuming the label is in the last column
X = df.iloc[:, :-1]  # All columns except the last
y = df.iloc[:, -1]   # The last column (target/output label)

# Perform 80/20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optional: Combine features and labels back into DataFrames if needed
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Encode features
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1, dtype=int)
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test)

# Save category mappings for later
encoded_feature_categories = encoder.categories_

Columns: 
['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'limit']


<ipython-input-28-a2f16f485e85>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip(" '\".") if isinstance(x, str) and not any(c in x for c in ('[', ']', '-inf')) else x)


In [29]:
from ganblr.models import GANBLR

# Initialise model
model = GANBLR()

# Fit the model on the cleaned training data
model.fit(X_train, y_train, k=0, epochs=100, batch_size=64)

warmup run:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7010 - loss: 0.5884  


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100: G_loss = 1.042413, G_accuracy = 0.715580, D_loss = 1.578001, D_accuracy = 0.389581


...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 97/100: G_loss = 0.462292, G_accuracy = 0.898551, D_loss = 1.421742, D_accuracy = 0.409966
Epoch 98/100: G_loss = 2.724999, G_accuracy = 0.898551, D_loss = 2.403559, D_accuracy = 0.405436


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 99/100: G_loss = 3.455568, G_accuracy = 0.896739, D_loss = 1.451096, D_accuracy = 0.561721


Epoch 100/100: G_loss = 5.977384, G_accuracy = 0.896739, D_loss = 2.838910, D_accuracy = 0.515289


In [30]:
# Generate synthetic data
synthetic_data = model.sample(size=len(X_train), verbose=1)
# synthetic_data = model.sample(1000), verbose=1)

# Convert to DataFrame with original column names
import pandas as pd
synthetic_df = pd.DataFrame(synthetic_data, columns=list(X_train.columns) + ['limit'])

# Display first few rows
synthetic_df.head()

  0%|          | 0/16 [00:00<?, ?it/s]

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,limit
0,a,missing,'(4.2075-inf)',l,gg,aa,bb,'(1.02-inf)',f,t,'(0.5-2.5]',f,g,'(-inf-105]','(-inf-492]',n
1,missing,'(-inf-38.96]','(4.2075-inf)',u,gg,missing,o,'(1.02-inf)',f,t,'(-inf-0.5]',t,g,missing,'(492-inf)',n
2,b,'(-inf-38.96]','(-inf-4.2075]',l,missing,m,n,'(1.02-inf)',t,f,'(-inf-0.5]',f,p,'(-inf-105]','(492-inf)',n
3,a,'(-inf-38.96]','(4.2075-inf)',missing,gg,r,ff,'(-inf-1.02]',f,t,'(0.5-2.5]',t,g,missing,'(-inf-492]',y
4,b,'(38.96-inf)','(-inf-4.2075]',missing,missing,ff,dd,'(-inf-1.02]',f,f,'(2.5-inf)',f,g,missing,'(-inf-492]',n


In [31]:
# Generate synthetic data matching real data size
synthetic_data = model.sample(size=len(X_train), verbose=0)

# Rebuild column names with label
synthetic_columns = list(X_train.columns) + ['limit']

# Create DataFrame
synthetic_df = pd.DataFrame(synthetic_data, columns=synthetic_columns)

In [32]:
print("Unique labels in y_train:", y_train.unique())
print("Unique labels in y_test:", y_test.unique())

Unique labels in y_train: ['n' 'y']
Unique labels in y_test: ['n' 'y']


In [33]:
print(f"Real: {X_train.assign(status=y_train).shape}, Synthetic: {synthetic_df.shape}")

Real: (552, 16), Synthetic: (552, 16)


In [34]:
def compare_all_distributions(real_df, synthetic_df, top_n=10):
    for column in real_df.columns:
        print(f"\n Feature: '{column}'")

        real_dist = real_df[column].value_counts(normalize=True).head(top_n)
        synthetic_dist = synthetic_df[column].value_counts(normalize=True).head(top_n)

        print("\nReal:")
        print(real_dist)

        print("\nSynthetic:")
        print(synthetic_dist)

        print("-" * 40)

# Run comparison
compare_all_distributions(X_train.assign(limit=y_train), synthetic_df)


 Feature: 'A1'

Real:
A1
b          0.682971
a          0.298913
missing    0.018116
Name: proportion, dtype: float64

Synthetic:
A1
a          0.351449
b          0.327899
missing    0.320652
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'A2'

Real:
A2
'(-inf-38.96]'    0.757246
'(38.96-inf)'     0.226449
missing           0.016304
Name: proportion, dtype: float64

Synthetic:
A2
'(-inf-38.96]'    0.411232
missing           0.331522
'(38.96-inf)'     0.257246
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'A3'

Real:
A3
'(-inf-4.2075]'    0.621377
'(4.2075-inf)'     0.378623
Name: proportion, dtype: float64

Synthetic:
A3
'(4.2075-inf)'     0.523551
'(-inf-4.2075]'    0.476449
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'A4'

Real:
A4
u          0.740942
y          0.246377
missing    0.010870
l          0.001812
Name: proportion, dtype: float64

Synthetic:
A4
y       

In [35]:
# Logistic Regression
accuracy_lr = model.evaluate(X_test.values, y_test.values, model='lr')
print(f"TSTR Accuracy (LR): {accuracy_lr:.4f}")

# Multi-Layer Perceptron
accuracy_mlp = model.evaluate(X_test.values, y_test.values, model='mlp')
print(f"TSTR Accuracy (MLP): {accuracy_mlp:.4f}")

# Random Forest
accuracy_rf = model.evaluate(X_test.values, y_test.values, model='rf')
print(f"TSTR Accuracy (RF): {accuracy_rf:.4f}")

from xgboost import XGBClassifier

# Create an XGBoost classifier instance
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Pass the instance directly
accuracy_xgbt = model.evaluate(X_test.values, y_test.values, model=xgb_model)
print(f"TSTR Accuracy (XGBT): {accuracy_xgbt:.4f}")

  0%|          | 0/16 [00:00<?, ?it/s]

TSTR Accuracy (LR): 0.8188


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


  0%|          | 0/16 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


TSTR Accuracy (RF): 0.7899


  0%|          | 0/16 [00:00<?, ?it/s]

TSTR Accuracy (XGBT): 0.8116


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:56:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [36]:
import pandas as pd
import arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

with open('dermatology.arff','r') as f:
  data = arff.load(f)
df = pd.DataFrame(data['data'], columns=[attr[0] for attr in data['attributes']])
df.replace('?', pd.NA, inplace=True)
df.rename(columns={'class': 'decision'}, inplace=True)
# Improved cleaning: only clean strings (ignoring interval range values)
df = df.applymap(lambda x: x.strip(" '\".") if isinstance(x, str) and not any(c in x for c in ('[', ']', '-inf')) else x)
print("Columns: ")
print(df.columns.tolist())
df.head(1)

# Separate features (X) and labels (y) — assuming the label is in the last column
X = df.iloc[:, :-1]  # All columns except the last
y = df.iloc[:, -1]   # The last column (target/output label)

# Perform 80/20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optional: Combine features and labels back into DataFrames if needed
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Encode features
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1, dtype=int)
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test)

# Save category mappings for later
encoded_feature_categories = encoder.categories_

Columns: 
['erythema', 'scaling', 'definite_borders', 'itching', 'koebner_phenomenon', 'polygonal_papules', 'follicular_papules', 'oral_mucosal_involvement', 'knee_and_elbow_involvement', 'scalp_involvement', 'family_history', 'melanin_incontinence', 'eosinophils_in_the_infiltrate', 'PNL_infiltrate', 'fibrosis_of_the_papillary_dermis', 'exocytosis', 'acanthosis', 'hyperkeratosis', 'parakeratosis', 'clubbing_of_the_rete_ridges', 'elongation_of_the_rete_ridges', 'thinning_of_the_suprapapillary_epidermis', 'spongiform_pustule', 'munro_microabcess', 'focal_hypergranulosis', 'disappearance_of_the_granular_layer', 'vacuolisation_and_damage_of_basal_layer', 'spongiosis', 'saw-tooth_appearance_of_retes', 'follicular_horn_plug', 'perifollicular_parakeratosis', 'inflammatory_monoluclear_inflitrate', 'band-like_infiltrate', 'Age', 'decision']


<ipython-input-36-151376ea6e69>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip(" '\".") if isinstance(x, str) and not any(c in x for c in ('[', ']', '-inf')) else x)


In [37]:
from ganblr.models import GANBLR

# Initialise model
model = GANBLR()

# Fit the model on the cleaned training data
model.fit(X_train, y_train, k=0, epochs=100, batch_size=64)

warmup run:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.2378 - loss: 1.8530


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100: G_loss = 2.344399, G_accuracy = 0.280822, D_loss = 1.287647, D_accuracy = 0.325482


...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 98/100: G_loss = 0.662381, G_accuracy = 0.993151, D_loss = 0.717960, D_accuracy = 0.582441


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 99/100: G_loss = 0.489946, G_accuracy = 0.993151, D_loss = 1.073883, D_accuracy = 0.391863


Epoch 100/100: G_loss = 0.991305, G_accuracy = 0.993151, D_loss = 1.075195, D_accuracy = 0.374732


In [38]:
# Generate synthetic data
synthetic_data = model.sample(size=len(X_train), verbose=1)
# synthetic_data = model.sample(1000), verbose=1)

# Convert to DataFrame with original column names
import pandas as pd
synthetic_df = pd.DataFrame(synthetic_data, columns=list(X_train.columns) + ['decision'])

# Display first few rows
synthetic_df.head()

  0%|          | 0/35 [00:00<?, ?it/s]

,erythema,scaling,definite_borders,itching,koebner_phenomenon,polygonal_papules,follicular_papules,oral_mucosal_involvement,knee_and_elbow_involvement,scalp_involvement,...,disappearance_of_the_granular_layer,vacuolisation_and_damage_of_basal_layer,spongiosis,saw-tooth_appearance_of_retes,follicular_horn_plug,perifollicular_parakeratosis,inflammatory_monoluclear_inflitrate,band-like_infiltrate,Age,decision
0,3,2,3,1,0,3,0,1,0,3,...,3,2,2,1,1,3,1,0,'(-inf-14]',2
1,2,1,3,0,2,3,0,1,2,1,...,0,3,1,3,2,1,3,0,'(14-inf)',3
2,3,3,0,1,1,3,3,2,0,0,...,0,0,3,0,3,0,1,3,'(-inf-14]',2
3,2,2,1,2,0,3,2,3,2,0,...,3,0,0,3,2,2,3,1,'(14-inf)',5
4,1,2,1,3,0,0,0,0,1,3,...,2,0,3,0,0,1,3,3,'(14-inf)',1


In [39]:
# Generate synthetic data matching real data size
synthetic_data = model.sample(size=len(X_train), verbose=0)

# Rebuild column names with label
synthetic_columns = list(X_train.columns) + ['decision']

# Create DataFrame
synthetic_df = pd.DataFrame(synthetic_data, columns=synthetic_columns)

In [40]:
print(f"Real: {X_train.assign(status=y_train).shape}, Synthetic: {synthetic_df.shape}")

Real: (292, 35), Synthetic: (292, 35)


In [41]:
def compare_all_distributions(real_df, synthetic_df, top_n=10):
    for column in real_df.columns:
        print(f"\n Feature: '{column}'")

        real_dist = real_df[column].value_counts(normalize=True).head(top_n)
        synthetic_dist = synthetic_df[column].value_counts(normalize=True).head(top_n)

        print("\nReal:")
        print(real_dist)

        print("\nSynthetic:")
        print(synthetic_dist)

        print("-" * 40)

# Run comparison
compare_all_distributions(X_train.assign(decision=y_train), synthetic_df)


 Feature: 'erythema'

Real:
erythema
2    0.599315
3    0.232877
1    0.157534
0    0.010274
Name: proportion, dtype: float64

Synthetic:
erythema
2    0.287671
3    0.263699
1    0.229452
0    0.219178
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'scaling'

Real:
scaling
2    0.541096
1    0.291096
3    0.143836
0    0.023973
Name: proportion, dtype: float64

Synthetic:
scaling
3    0.291096
2    0.263699
1    0.236301
0    0.208904
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'definite_borders'

Real:
definite_borders
2    0.469178
1    0.236301
0    0.167808
3    0.126712
Name: proportion, dtype: float64

Synthetic:
definite_borders
3    0.294521
0    0.267123
1    0.226027
2    0.212329
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'itching'

Real:
itching
0    0.325342
2    0.280822
3    0.215753
1    0.178082
Name: proportion, dtype: float64

Synthetic:
itching


In [42]:
# Logistic Regression
accuracy_lr = model.evaluate(X_test.values, y_test.values, model='lr')
print(f"TSTR Accuracy (LR): {accuracy_lr:.4f}")

# Multi-Layer Perceptron
accuracy_mlp = model.evaluate(X_test.values, y_test.values, model='mlp')
print(f"TSTR Accuracy (MLP): {accuracy_mlp:.4f}")

# Random Forest
accuracy_rf = model.evaluate(X_test.values, y_test.values, model='rf')
print(f"TSTR Accuracy (RF): {accuracy_rf:.4f}")

from xgboost import XGBClassifier

# Create an XGBoost classifier instance
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Pass the instance directly
accuracy_xgbt = model.evaluate(X_test.values, y_test.values, model=xgb_model)
print(f"TSTR Accuracy (XGBT): {accuracy_xgbt:.4f}")

  0%|          | 0/35 [00:00<?, ?it/s]

TSTR Accuracy (LR): 0.8514


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


  0%|          | 0/35 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


TSTR Accuracy (RF): 0.7432


  0%|          | 0/35 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:24:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


TSTR Accuracy (XGBT): 0.8784


In [43]:
import pandas as pd
import arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

with open('letter-recog.arff','r') as f:
  data = arff.load(f)
df = pd.DataFrame(data['data'], columns=[attr[0] for attr in data['attributes']])
df.replace('?', pd.NA, inplace=True)
df.rename(columns={'class': 'size'}, inplace=True)
# Improved cleaning: only clean strings (ignoring interval range values)
df = df.applymap(lambda x: x.strip(" '\".") if isinstance(x, str) and not any(c in x for c in ('[', ']', '-inf')) else x)
print("Columns: ")
print(df.columns.tolist())
df.head(1)

# Separate features (X) and labels (y) — assuming the label is in the last column
X = df.iloc[:, :-1]  # All columns except the last
y = df.iloc[:, -1]   # The last column (target/output label)

# Perform 80/20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optional: Combine features and labels back into DataFrames if needed
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Encode features
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1, dtype=int)
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test)

# Save category mappings for later
encoded_feature_categories = encoder.categories_

<ipython-input-43-9aad27b4be09>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip(" '\".") if isinstance(x, str) and not any(c in x for c in ('[', ']', '-inf')) else x)


Columns: 
['x-box', 'y-box', 'width', 'high', 'onpix', 'x-bar', 'y-bar', 'x2bar', 'y2bar', 'xybar', 'x2ybr', 'xy2br', 'x-ege', 'xegvy', 'y-ege', 'yegvx', 'size']


In [44]:
from ganblr.models import GANBLR

# Initialise model
model = GANBLR()

# Fit the model on the cleaned training data
model.fit(X_train, y_train, k=0, epochs=100, batch_size=64)

warmup run:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1713 - loss: 3.0720


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100: G_loss = 3.707972, G_accuracy = 0.638125, D_loss = 1.079666, D_accuracy = 0.599102


...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 98/100: G_loss = 2.413457, G_accuracy = 0.892500, D_loss = 1.870581, D_accuracy = 0.570742


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 99/100: G_loss = 1.813979, G_accuracy = 0.892625, D_loss = 1.673115, D_accuracy = 0.510547


Epoch 100/100: G_loss = 1.837056, G_accuracy = 0.892875, D_loss = 1.555766, D_accuracy = 0.511602


In [45]:
# Generate synthetic data
synthetic_data = model.sample(size=len(X_train), verbose=1)
# synthetic_data = model.sample(1000), verbose=1)

# Convert to DataFrame with original column names
import pandas as pd
synthetic_df = pd.DataFrame(synthetic_data, columns=list(X_train.columns) + ['size'])

# Display first few rows
synthetic_df.head()

  0%|          | 0/17 [00:00<?, ?it/s]

,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx,size
0,'(1.5-2.5]',All,'(0.5-3.5]','(9.5-11.5]','(2.5-5.5]','(12.5-inf)','(8.5-9.5]','(2.5-3.5]','(2.5-3.5]','(14.5-inf)','(4.5-5.5]','(13.5-14.5]','(-inf-0.5]','(6.5-7.5]','(0.5-1.5]','(7.5-8.5]',J
1,'(4.5-inf)',All,'(7.5-9.5]','(-inf-8.5]','(0.5-1.5]','(3.5-4.5]','(5.5-6.5]','(2.5-3.5]','(7.5-8.5]','(6.5-7.5]','(9.5-10.5]','(5.5-6.5]','(4.5-5.5]','(7.5-8.5]','(5.5-7.5]','(7.5-8.5]',X
2,'(4.5-inf)',All,'(7.5-9.5]','(-inf-8.5]','(5.5-9.5]','(7.5-8.5]','(10.5-11.5]','(5.5-6.5]','(3.5-4.5]','(10.5-11.5]','(14.5-inf)','(3.5-4.5]','(2.5-3.5]','(9.5-10.5]','(2.5-3.5]','(5.5-6.5]',F
3,'(2.5-4.5]',All,'(0.5-3.5]','(-inf-8.5]','(-inf-0.5]','(3.5-4.5]','(5.5-6.5]','(11.5-12.5]','(3.5-4.5]','(3.5-4.5]','(7.5-8.5]','(7.5-8.5]','(6.5-7.5]','(8.5-9.5]','(1.5-2.5]','(9.5-10.5]',U
4,'(0.5-1.5]',All,'(3.5-7.5]','(9.5-11.5]','(5.5-9.5]','(-inf-0.5]','(9.5-10.5]','(2.5-3.5]','(6.5-7.5]','(10.5-11.5]','(4.5-5.5]','(8.5-9.5]','(3.5-4.5]','(9.5-10.5]','(7.5-inf)','(6.5-7.5]',F


In [46]:
# Generate synthetic data matching real data size
synthetic_data = model.sample(size=len(X_train), verbose=0)

# Rebuild column names with label
synthetic_columns = list(X_train.columns) + ['size']

# Create DataFrame
synthetic_df = pd.DataFrame(synthetic_data, columns=synthetic_columns)

In [47]:
print(f"Real: {X_train.assign(size=y_train).shape}, Synthetic: {synthetic_df.shape}")

Real: (16000, 17), Synthetic: (16000, 17)


In [48]:
def compare_all_distributions(real_df, synthetic_df, top_n=10):
    for column in real_df.columns:
        print(f"\n Feature: '{column}'")

        real_dist = real_df[column].value_counts(normalize=True).head(top_n)
        synthetic_dist = synthetic_df[column].value_counts(normalize=True).head(top_n)

        print("\nReal:")
        print(real_dist)

        print("\nSynthetic:")
        print(synthetic_dist)

        print("-" * 40)

# Run comparison
compare_all_distributions(X_train.assign(size=y_train), synthetic_df)


 Feature: 'x-box'

Real:
x-box
'(2.5-4.5]'     0.432500
'(4.5-inf)'     0.353312
'(1.5-2.5]'     0.143687
'(0.5-1.5]'     0.063812
'(-inf-0.5]'    0.006687
Name: proportion, dtype: float64

Synthetic:
x-box
'(4.5-inf)'     0.283250
'(2.5-4.5]'     0.248438
'(1.5-2.5]'     0.204937
'(0.5-1.5]'     0.191188
'(-inf-0.5]'    0.072188
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'y-box'

Real:
y-box
All    1.0
Name: proportion, dtype: float64

Synthetic:
y-box
All    1.0
Name: proportion, dtype: float64
----------------------------------------

 Feature: 'width'

Real:
width
'(3.5-7.5]'     0.684063
'(0.5-3.5]'     0.183062
'(7.5-9.5]'     0.104188
'(9.5-inf)'     0.018937
'(-inf-0.5]'    0.009750
Name: proportion, dtype: float64

Synthetic:
width
'(3.5-7.5]'     0.325500
'(0.5-3.5]'     0.259625
'(7.5-9.5]'     0.241313
'(9.5-inf)'     0.113625
'(-inf-0.5]'    0.059937
Name: proportion, dtype: float64
----------------------------------------

 Featu

In [49]:
# Logistic Regression
accuracy_lr = model.evaluate(X_test.values, y_test.values, model='lr')
print(f"TSTR Accuracy (LR): {accuracy_lr:.4f}")

# Multi-Layer Perceptron
accuracy_mlp = model.evaluate(X_test.values, y_test.values, model='mlp')
print(f"TSTR Accuracy (MLP): {accuracy_mlp:.4f}")

# Random Forest
accuracy_rf = model.evaluate(X_test.values, y_test.values, model='rf')
print(f"TSTR Accuracy (RF): {accuracy_rf:.4f}")

from xgboost import XGBClassifier

# Create an XGBoost classifier instance
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Pass the instance directly
accuracy_xgbt = model.evaluate(X_test.values, y_test.values, model=xgb_model)
print(f"TSTR Accuracy (XGBT): {accuracy_xgbt:.4f}")

  0%|          | 0/17 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


TSTR Accuracy (LR): 0.6092


  0%|          | 0/17 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


TSTR Accuracy (RF): 0.4407


  0%|          | 0/17 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:00:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


TSTR Accuracy (XGBT): 0.5100
